In [ ]:
# Pipeline: Pun -> 2 sense def -> Reverse dictionary -> all keywords -> generation

## set path here
path="path/to/folder"
openaikey="xxxxxx"

In [1]:
!git clone https://github.com/thunlp/WantWords.git
!pip install torch
!pip install pytorch_transformers

fatal: destination path 'WantWords' already exists and is not an empty directory.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/anaconda/envs/env_pytorch/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/anaconda/envs/env_pytorch/bin/python -m pip install --upgrade pip' command.


In [3]:
## Sense keys for a word from the dataset
import pandas as pd 
def get_senses(pun):
  df = pd.read_csv(path+"Dataset1.11.csv")
  for index, row in df.iterrows():
    if row['Pun']==pun:
      senseKeys=[row['Sense 1'],row['sense 2']]
  # senseKeys=[row['Sense 1'],row['sense 2']]
  return senseKeys

In [4]:
### Sense definitions from the pun word
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def sense_def(pun):
  senseKeys= get_senses(pun)
  sense_definitions=[]
  for senseKey in senseKeys:
    m=wn.lemma_from_key(senseKey).synset().definition()
    sense_definitions.append(m)
  return sense_definitions

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/development/nihars/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/development/nihars/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/development/nihars/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
#### Reverse Dicitonary

import torch
import torch.nn as nn
import torch.optim as optim
import torch, gc, json, os, string, re, requests, hashlib, urllib.parse 
import numpy as np
from datetime import datetime
from pytorch_transformers import *

path_loc = path + "Code/WantWords/"
%cd $path_loc
ani= torch.load("../website_RD_data/1rsl_saved.model", map_location=lambda storage, loc: storage)
tokenizer_class = BertTokenizer
tokenizer_Ch = tokenizer_class.from_pretrained('bert-base-chinese')
tokenizer_En = tokenizer_class.from_pretrained('bert-base-uncased')
(_, (_, label_size, _, _), (word2index_en, index2word_en, index2sememe, index2lexname, index2rootaffix)) = np.load("../website_RD_data/"+ 'data_inUse1_en.npy', allow_pickle=True)
device = torch.device('cpu')
words_t = torch.tensor(np.array([0]))
def label_multihot(labels, num):
    sm = np.zeros((len(labels), num), dtype=np.float32)
    for i in range(len(labels)):
        for s in labels[i]:
            if s >= num:
                break
            sm[i, s] = 1
    return sm



def word2feature(dataset, word_num, feature_num, feature_name):
    max_feature_num = max([len(instance[feature_name]) for instance in dataset])
    ret = np.zeros((word_num, max_feature_num), dtype=np.int64)
    ret.fill(feature_num)
    for instance in dataset:
        if ret[instance['word'], 0] != feature_num: 
            continue # this target_words has been given a feature mapping, because same word with different definition in dataset
        feature = instance[feature_name]
        ret[instance['word'], :len(feature)] = np.array(feature)
    return torch.tensor(ret, dtype=torch.int64, device=device)
    
def mask_noFeature(label_size, wd2fea, feature_num):
    mask_nofea = torch.zeros(label_size, dtype=torch.float32, device=device)
    for i in range(label_size):
        feas = set(wd2fea[i].detach().cpu().numpy().tolist())-set([feature_num])
        if len(feas)==0:
            mask_nofea[i] = 1
    return mask_nofea
    
(data_train_idx, data_dev_idx, data_test_500_seen_idx, data_test_500_unseen_idx, data_defi_c_idx, data_desc_c_idx) = np.load("../website_RD_data/" + 'data_inUse2_en.npy', allow_pickle=True)
data_all_idx = data_train_idx + data_dev_idx + data_test_500_seen_idx + data_test_500_unseen_idx + data_defi_c_idx
index2word_en = np.array(index2word_en)
print('-------------------------2')
sememe_num = len(index2sememe)
wd2sem = word2feature(data_all_idx, label_size, sememe_num, 'sememes')
wd_sems_ = label_multihot(wd2sem, sememe_num)
wd_sems_ = torch.from_numpy(np.array(wd_sems_)).to(device) 
lexname_num = len(index2lexname)
wd2lex = word2feature(data_all_idx, label_size, lexname_num, 'lexnames') 
wd_lex = label_multihot(wd2lex, lexname_num)
wd_lex = torch.from_numpy(np.array(wd_lex)).to(device)
rootaffix_num = len(index2rootaffix)
wd2ra = word2feature(data_all_idx, label_size, rootaffix_num, 'root_affix') 
wd_ra = label_multihot(wd2ra, rootaffix_num)
wd_ra = torch.from_numpy(np.array(wd_ra)).to(device)
mask_s_ = mask_noFeature(label_size, wd2sem, sememe_num)
mask_l = mask_noFeature(label_size, wd2lex, lexname_num)
mask_r = mask_noFeature(label_size, wd2ra, rootaffix_num)


MODE_en = 'rsl'
itemsPerCol = 20
GET_NUM = 100
NUM_RESPONSE = 500
def Score2Hexstr(score, maxsc):
    thr = maxsc/1.5
    l = len(score)
    ret = ['00']*l
    for i in range(l):
        res = int(200*(score[i] - thr)/thr)
        if res>15:
            ret[i] = hex(res)[2:]
        else:
            break
    return ret

def keywords(sentences):
  output=[]
  for description in sentences:
  
    RD_mode='EE'
    with torch.no_grad():
      def_words = re.sub('[%s]' % re.escape(string.punctuation), ' ', description)
      def_words =def_words.lower()
      def_words = def_words.strip().split()
      def_word_idx = []
      # print("def words:", def_words)
      if len(def_words) > 0:
        for def_word in def_words:
            if def_word in word2index_en:
              def_word_idx.append(word2index_en[def_word])
            else:
              # print("hi")
              def_word_idx.append(word2index_en['<OOV>'])
        # print("def_word_idx: ", def_word_idx, print(word2index_en['<OOV>']),set(def_word_idx),{word2index_en['<OOV>']})
        x_len = len(def_word_idx)
        if set(def_word_idx)=={word2index_en['<OOV>']}:
            x_len = 1
        if x_len==1:
            if def_word_idx[0]>1:
                #词向量找相关词，排序后，如果在WordNet的synset里，则对应的同义词的分数乘以2？
                score = ((model_en.embedding.weight.data).mm((model_en.embedding.weight.data[def_word_idx[0]]).unsqueeze(1))).squeeze(1)
                if RD_mode=='EE': #当EE的时候，排除自身，CE的时候自身是最准确的，不排除。
                    score[def_word_idx[0]] = -10.
                score[np.array(index2synset_en[def_word_idx[0]])] *= 2
                sc, indices = torch.sort(score, descending=True)
                predicted = indices[:NUM_RESPONSE].detach().cpu().numpy()
                
                score = sc[:NUM_RESPONSE].detach().numpy()
                maxsc = sc[0].detach().item()
                s2h = Score2Hexstr(score, maxsc)
            else:
                predicted= []
                ret = {'error': 1} # 字符无法识别
        else:
            defi = '[CLS] ' + description
            def_word_idx = tokenizer_En.encode(defi)[:60]
            def_word_idx.extend(tokenizer_En.encode('[SEP]'))
            definition_words_t = torch.tensor(np.array(def_word_idx), dtype=torch.int64, device=device)
            definition_words_t = definition_words_t.unsqueeze(0) # batch_size = 1
            score = ani('test', x=definition_words_t, w=words_t, ws=wd_sems_, wl=wd_lex, wr=wd_ra, msk_s=mask_s_, msk_l=mask_l, msk_r=mask_r, mode=MODE_en)
            sc, indices = torch.sort(score, descending=True)
            predicted = indices[0, :NUM_RESPONSE].detach().cpu().numpy()
            score = sc[0, :NUM_RESPONSE].detach().numpy()
            maxsc = sc[0, 0].detach().item()
            s2h = Score2Hexstr(score, maxsc)
              
      else:
        print("hiiii")
        predicted= []
        ret = {'error': 0} # 输入为空
    # print("predicted: ", predicted)
    if len(predicted)>0:
        res = index2word_en[predicted]
        # print("res: ",res)
        ret = [] # 不能以字典形式返回，因为字典是无序的，这里需要用列表来保持顺序。
        cn = -1
        if RD_mode == "EE":
            def_words = set(def_words)
            for wd in res:
                cn += 1
                if len(wd)>1 and (wd not in def_words):
                    # ret.append(wd_data_en[wd]) # wd_data_en[wd] = {'word': word, 'definition':defis, 'POS':['n']}]
                    # ret[len(ret)-1]['c'] = s2h[cn]
                    try:
                        ret.append(wd_data_en[wd]) # wd_data_en[wd] = {'word': word, 'definition':defis, 'POS':['n']}]
                        ret[len(ret)-1]['c'] = s2h[cn]
                    except:
                        continue
        else:
            for wd in res:
                cn += 1
                if len(wd)>1:
                    try:
                        ret.append(wd_data_en[wd]) # wd_data_en[wd] = {'word': word, 'definition':defis, 'POS':['n']}]
                        ret[len(ret)-1]['c'] = s2h[cn]
                    except:
                        continue
    output.append(res[0:8])
  return output

    # json.dumps(ret,ensure_ascii=False),content_type="application/json,charset=utf-8")
  

/home/development/nihars/anirudh/Anirudh-UCLA/Code/WantWords


/home/development/nihars/.local/lib/python3.7/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'model_en.Encoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/development/nihars/.local/lib/python3.7/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/development/nihars/.local/lib/python3.7/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by 

-------------------------2


In [ ]:
#### Getting related keywords for TFIDF

!pip install rake-nltk
from rake_nltk import Rake
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import os
oneb=[]
# paths=[path+"Data/1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled/",path+"Data/1-billion-word-language-modeling-benchmark-r13output/heldout-monolingual.tokenized.shuffled/"]
# for path in paths:
#   print(len(os.listdir(path)))
#   for filename in os.listdir(path):
#     file=open(path+filename,"r")
#     try:
#       oneb.append(file.read())
#     except:
#       print(filename)
import pickle
with open(path+'oneb.p', 'rb') as fp:
    data = pickle.load(fp)
oneb=data



def generate_sentences(word):
  sentences=[]
  # word='contact'
  for article in oneb:
    for sentence in article.split('.'):
      if word in sentence:
        sentences.append(sentence)

  # print(word, len(sentences))
  # print(sentences)
  return sentences


# master={}

# for article in oneb:
#   for sentence in article.split('.'):
#     for word in sentence:
#       if word in master:
#         master[word]+=1
#       else:
#         master[word]=1

# try:
#     import cPickle as pickle
# except ImportError:  # Python 3.x
#     import pickle
# # print(path,len(oneb))
# with open(path+'master.p', 'wb') as fp:
#     pickle.dump(master, fp, protocol=pickle.HIGHEST_PROTOCOL)

def find_number(wordx):
  if wordx in master:
    return master[wordx]
  else:
    return 0
with open(path+'master.p', 'rb') as fp:
    data1 = pickle.load(fp)
master=data1

from scipy import spatial
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
model_sentence = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def refine_sentences(sentences, senseDef):
  sentences_refined=[]
  for sentence in sentences:
    sentence=sentence.strip()
    embeddings = model_sentence.encode([sentence, senseDef])
    result = 1 - spatial.distance.cosine(embeddings[0], embeddings[1])
    # print("result: ", result)
    if result> 0.10:
      sentences_refined.append(sentence)
  # print("sentence_refined: ", sentences_refined[0:5])
  return sentences_refined



def final_keywords(interim_keywords, senseDef):
  final_keywords=[]
  for words in interim_keywords:
    final_keyword=[]
    # print("words:", words)
    for word in words:
      sentences = generate_sentences(word)
      # print("index: ",interim_keywords.index(words) )
      # sentences_refined= refine_sentences(sentences, senseDef[interim_keywords.index(words)])
      r = Rake(max_length=1)
      r.extract_keywords_from_sentences(sentences)
      # print(sentences[0:5])
      keywordsa= r.get_ranked_phrases()
      freq = {}
      for item in keywordsa:
        if (item in freq):
            freq[item] += 1
        else:
            freq[item] = 1
      freq= sorted(freq.items(), key=lambda x: x[1], reverse=True)
      freq1={}
      for freqz in freq[:100]:
        num = find_number(freqz[0]) # freqz[0] -> word , freqz[1] -> number
        freq1[freqz[0]]=freqz[1]/(num+1)
      freq1= sorted(freq1.items(), key=lambda x: x[1], reverse=True)
      checker=0
      for freqz in freq1[:10]:
        if freqz[0] != word and freqz[0] not in stop_words:
          final_keyword.append(freqz[0])
          checker=checker+1
        if checker>=5:
          break

    final_keywords.append(final_keyword)
  return final_keywords

  

In [ ]:
#### Getting related keywords for Word2Vec

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

with open(path+"LM_model_distance.txt") as file:
  lines= file.readlines()

def final_keywords(interim_keywords):
  final_keywords=[]
  for words in interim_keywords:
    final_keyword=[]
    for word in words:
      for linex in lines:
        line = linex.split(" ")
        if line[0]==word:
          
          for i in range(2,120,2):
            if i not in stop_words and len(final_keyword)<=30 and i != word:
              final_keyword.append(line[i])
    final_keywords.append(final_keyword)
  return final_keywords

In [ ]:
!python3 -m pip install openai
import os
import openai
openai.api_key = os.getenv(openaikey)
!export OPENAI_API_KEY= openaikey
openai.api_key = openaikey
engine_name = "davinci-instruct-beta"
def final_keywords(keywords, pun):
    flist=[]

    for klist in keywords:
        fflist=[]
        for word in klist:
            prompt = '''Generate seven keywords for computer and internet: mouse, hardware, fast, smart, email, technology, software.
            Generate seven keywords for '''+ word+" and "+pun +':'
            response = openai.Completion.create(
            engine= engine_name,
            prompt=prompt,
            temperature=0.2,
            max_tokens=15,
            top_p=0.95,
            frequency_penalty=0.5,
            presence_penalty=0.5
            )
            # print("-----"+keyword)
            # print(response['choices'][0]['text'])
            nlp = response['choices'][0]['text']
            nlp=nlp.split(".")[0]
            nlp=nlp.split(",")
            # cfflist.append(nlp)
            nlp = [a.strip(" ") for a in nlp]
            if word in nlp:
                nlp.remove(word)
            # print(nlp)
            fflist=fflist+nlp
        # print("fflist: ", fflist)
        flist.append(fflist)
    # print(flist)
    return flist






In [7]:
###Generating candidate sentences

!pip install simplet5
from simplet5 import SimpleT5
import random
# from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel

# device = torch.device('cuda')

model_t5 = SimpleT5()
# output_dir=path+"CandidateGeneration/GPT2/wo_sampling/epoch3"
# model_gpt2 = GPT2LMHeadModel.from_pretrained(output_dir)
# tokenizer_gpt2 = GPT2Tokenizer.from_pretrained(output_dir)
# # def load_models():
# device = torch.device("cuda:1")
# model_gpt2.to(device)
model_t5.from_pretrained(model_type="t5", model_name="t5-base")
model_t5.load_model("t5",path+"simplet5-epoch-5-train-loss-1.4723", use_gpu=True)
  

def generate_candidates1(x, pun, flag="mix"):
  sense1=x[0]
  sense2=x[1]
  candidates=[]

  if flag=="one" or flag=="mix":
    for i in range(50):
      a = random.randint(0,len(sense1)-1)
      b = random.randint(0,len(sense2)-1)
      if i<16:
        text="generate sentence: "+str(pun)+", "+str(sense1[a])+", "+str(sense2[b])
      elif i<37:
        text="generate sentence: "+str(sense1[a])+", "+str(pun)+", "+str(sense2[b])
      else:
        text="generate sentence: "+str(sense1[a])+", "+str(sense2[b])+", "+str(pun)

      candidates.append([model_t5.predict(text)[0],text])
  if flag=="two" or flag=="mix":
    for i in range(50):
      a = random.randint(0,len(sense1)-1)
      b = random.randint(0,len(sense1)-1)
      c = random.randint(0,len(sense2)-1)
      d = random.randint(0,len(sense2)-1)
      if i<0:
        text="generate sentence: "+str(pun)+", "+str(sense1[a])+", "+str(sense1[b])+", "+str(sense2[c])+", "+str(sense2[d])
      elif i<8:
        text="generate sentence: "+str(sense1[a])+", "+str(sense1[b])+", "+str(pun)+", "+str(sense2[c])+", "+str(sense2[d])
      else:
        text="generate sentence: "+str(sense1[a])+", "+str(sense1[b])+", "+str(sense2[c])+", "+str(sense2[d])+", "+str(pun)
      candidates.append([model_t5.predict(text)[0],text])
  if flag=="two" or flag=="mix":
    for i in range(75):
      a = random.randint(0,len(sense1)-1)
      b = random.randint(0,len(sense1)-1)
      c = random.randint(0,len(sense2)-1)
      d = random.randint(0,len(sense2)-1)
      if i<0:
        text="generate sentence: "+str(pun)+", "+str(sense1[a])+", "+str(sense1[b])+", "+str(sense2[c])
      elif i<15:
        text="generate sentence: "+str(sense1[a])+", "+str(sense1[b])+", "+str(pun)+", "+str(sense2[c])
      elif i<40:
        text="generate sentence: "+str(sense1[a])+", "+str(sense2[d])+", "+str(sense2[c])+", "+str(pun)
      else:
        text="generate sentence: "+str(sense1[a])+", "+str(sense1[b])+", "+str(sense2[c])+", "+str(pun)
      candidates.append([model_t5.predict(text)[0],text])

  return candidates



Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/anaconda/envs/env_pytorch/bin/python -m pip install --upgrade pip' command.


Global seed set to 42


In [8]:
flag = "two"

In [9]:
##Data refinement process

import pickle
import re
df_aloha = pd.read_csv(path+"Dataset1.11.csv")
aloha=df_aloha['Pun'].to_list()
def Refine(keywords_lists, word , num=100):
    aa=[]
    # print("num:",num)
    for keyword_list in keywords_lists:
        a=[]
        for item in keyword_list:
            item=item.strip()
            item=item.strip(" ")
            item= re.sub(r'\W+', '', item)
            item = ''.join([i for i in item if not i.isdigit()])
            if item != word and item not in stop_words and item not in a and item not in aloha and len(item)>0:
                a.append(item)
        if len(a)>num:
            aa.append(a[:num])
        else:
            aa.append(a)
    return aa

In [10]:
words=["addresses"]

In [12]:
for word in words:
    try:
        print("Pun word: ", word)
        senseDefinitions = sense_def(word)
        print("sense definitions: ",senseDefinitions)
        ReverseDicKeywords = keywords(senseDefinitions)
        print("ReverseDicKeywords: ",ReverseDicKeywords)
        ReverseDicKeywords_refine = Refine(ReverseDicKeywords, word,5 )
        print("ReverseDicKeywords_refine: ",ReverseDicKeywords_refine)
        finalKeywords= final_keywords(ReverseDicKeywords_refine, word)
        print("finalKeywords: ",finalKeywords)
        finalKeywords_refine= Refine(finalKeywords, word)
        print("finalKeywords_refine",finalKeywords_refine)
        candidateSentences1 = generate_candidates1(finalKeywords_refine, word, flag)
        print("Candidate sentences 1: ", candidateSentences1[0:5])
        with open(path+str(word_temp)+'_.p', 'wb') as fx:
            pickle.dump(candidateSentences1, fx, protocol=pickle.HIGHEST_PROTOCOL)

    except Exception as e:
        print("Error: ", word, e)

Pun word:  adresses
Error:  adresses local variable 'senseKeys' referenced before assignment
Pun word:  bugs
sense definitions:  ['general term for any insect or similar creeping or crawling invertebrate', 'a fault or defect in a computer program, system, or machine']
ReverseDicKeywords_refine:  [['spider', 'cockroach', 'termite', 'snail', 'earthworm'], ['flaw', 'checksum', 'error', 'misstep', 'problem']]
finalKeywords_refine [['spiders', 'said', 'one', 'cockroaches', 'rats', 'mice', 'termites', 'insects', 'company', 'found', 'ants', 'snails', 'pace', 'slugs', 'shell', 'earthworms', 'soil', 'species'], ['flawed', 'flaws', 'said', 'one', 'flawless', 'method', 'woke', 'verify', 'user', 'tool', 'terrorism', 'terrorists', 'terror', 'war', 'missteps', 'series', 'people', 'solve']]
Candidate sentences 1:  [["A user can verify if he has spiders cockroaches bugs or a person's facebook status.", 'generate sentence: cockroaches, spiders, bugs, user, verify'], ['What do insects and rats have in c

In [5]:
import pickle
with open(path+'hit.p', 'rb') as fp:
    data1 = pickle.load(fp)
data1

[]

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast
import matplotlib
from scipy.special import softmax
device = torch.device("cuda:1")

import pickle
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-large-uncased',return_dict=False)

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-large-uncased')

max_seq_len = 20
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(1024,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id, attention_mask=mask)
      
      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)

      return x

# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)

#load weights of best model
path1 = path+'saved_weights_colbert.pt'
model.load_state_dict(torch.load(path1))

with torch.no_grad():
  preds = model(infer_seq.to(device), infer_mask.to(device))
  preds = preds.detach().cpu().numpy()
logits = softmax(preds, axis = 1)

score_dict={}
for text, score in zip(infer_text,logits[:,1]):
    score_dict[text] = score

ranked_dict = {k: v for k, v in sorted(score_dict.items(), key=lambda item: item[1], reverse = True)}
for item in ranked_dict:
    if word.strip('_new') in item:
        print(item, ranked_dict[item], sep = '\t')